# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_BASE_URL = "http://localhost:11434/v1"
model=MODEL_GPT

In [3]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")

In [4]:
#system prompt
system_prompt = """
You are a senior technical assistant who provides accurate, clear, and practical answers to technical questions in software engineering and related fields.
Prioritize correctness over speed, state assumptions explicitly, and avoid speculation or hallucination.
Begin with a direct answer, then explain step by step when useful, using minimal, idiomatic, and runnable code examples where appropriate.
When debugging or comparing solutions, explain the underlying causes, trade-offs, and how to verify results.
Ask clarifying questions only when necessary and adapt the level of detail to the user's apparent expertise.

"""

In [5]:
# Get gpt-4o-mini to answer, with streaming

openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


def chat(user_message, history, model):
    # history is a list of [user, assistant] pairs
    messages = [{"role": "system", "content": system_prompt}]

    if history:
        for item in history:
            if isinstance(item, (list, tuple)) and len(item) == 2:
                u, a = item
                if u: messages.append({"role": "user", "content": u})
                if a: messages.append({"role": "assistant", "content": a})
            elif isinstance(item, dict) and "role" in item and "content" in item:
                messages.append({"role": item["role"], "content": item["content"]})

    messages.append({"role": "user", "content": user_message})

    if model == MODEL_GPT:
        yield from call_openai(messages)
    elif model == MODEL_LLAMA:
        text = call_ollama(messages)
        yield text
    else:
        return f"Unknown model: {model}"

def call_openai(messages):
        resp = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=messages,
            stream = True
        )
        
        response = ""
        
        for chunk in resp:
            delta = chunk.choices[0].delta.content
            if delta:
                response += delta
                yield response

# Get Llama 3.2 to answer

def call_ollama(messages):
    resp = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages,
    )
    
    return resp.choices[0].message.content


In [6]:
model_selector = gr.Radio(
    choices=[MODEL_GPT, MODEL_LLAMA],
    value=MODEL_GPT,
    label="Model"
)

chat = gr.ChatInterface(
    fn=chat,
    additional_inputs=model_selector,
    title="Technical Assistant",
)
chat.launch()